<h1 align="center"> Sensordatenfusion Tutorium 02 </h1>

<h3 align="center"> Erstellen des Transitionsmodells </h3>

In [16]:
import numpy as np
from scipy.linalg import block_diag

from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import (TimeVariantModel)
from stonesoup.models.transition.linear import LinearGaussianTransitionModel

Wir benötigen unser Transitionsmodell, um letztendlich basierend auf unseren alten Kenntnissen eine
Prädiktion für die Zukunft erstellen zu können. Hierfür gibt es verschiedene Ansätze, 
im Folgenden werden wir mit dem $Piecewise Constant White Acceleration Model$ arbeiten:

$\Delta T$ = $t_k$ - $t_{k-1}$

$F_{k|k-1}$ = $\begin{bmatrix}
  1 & \Delta T_k  \\
  0 & 1  \\
\end{bmatrix}$ 

$D_{k|k-1}$ = $\Sigma^2_k$ * $\begin{bmatrix}
  \frac{1}{4}\Delta T^4_k 1 & \frac{1}{2}\Delta T^3_k 1  \\
  \frac{1}{2}\Delta T^3_k 1 & \Delta T^2_k 1  \\
\end{bmatrix}$

Wichtig für das Verständnis: $\Sigma^2_k$ ist hierbei kein Summenzeichen, sondern ein regulärer Vorfaktor!


In [17]:
class PCWAModel(LinearGaussianTransitionModel, TimeVariantModel):

    def matrix(self, timedelta=5, **kwargs):
        delta_t = timedelta
        F = np.array([[1, delta_t], [0, 1]])
        return block_diag(F, F)

    def covar(self, timedelta=5, **kwargs):
        delta_t = timedelta
        Sigma = 5.0

        covar = np.array([[np.power(delta_t, 4) / 4,
                           np.power(delta_t, 3) / 2],
                          [np.power(delta_t, 3) / 2,
                           np.power(delta_t, 2)]]) * np.power(Sigma, 2)

        covar = block_diag(covar, covar)

        return CovarianceMatrix(covar)

In [18]:
transition_model = PCWAModel()
print( transition_model.matrix())
print("D:")
transition_model.covar()


[[1 5 0 0]
 [0 1 0 0]
 [0 0 1 5]
 [0 0 0 1]]
D:


CovarianceMatrix([[3906.25, 1562.5 ,    0.  ,    0.  ],
                  [1562.5 ,  625.  ,    0.  ,    0.  ],
                  [   0.  ,    0.  , 3906.25, 1562.5 ],
                  [   0.  ,    0.  , 1562.5 ,  625.  ]])